<a href="https://colab.research.google.com/github/Wastfulajr/NetGuard/blob/Michael/PracticeAndRandomization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data initialization

In [1]:
import pandas as pd
import requests
import zipfile
import numpy as np #working
import io
#This is me
#THIS CODE JUST LOADS DATA INTO df
# Download and load the dataset into one DataFrame
dfs = []
FILE_DOWNLOAD_URL = "http://cicresearch.ca/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/GeneratedLabelledFlows.zip"

response = requests.get(FILE_DOWNLOAD_URL)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    for file in z.namelist()[1:]:  # skip the first entry if it's a folder
        with z.open(file) as f:
            dfs.append(pd.read_csv(f, encoding='cp1252'))

# Combine all CSVs into a single DataFrame
df = pd.concat(dfs, ignore_index=True)




/tmp/ipython-input-4080956491.py:16: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(f, encoding='cp1252'))


# Practice/ useful commands

In [ ]:
print(df.shape)
#should be around 3119345, 85) confirms dataset is loaded

(3119345, 85)


In [ ]:
#sorts by port and protocol together
df_sorted = df.sort_values(by=[" Destination Port", " Protocol"], ascending=[False, False])
print("\nSorted dataset preview:")
print(df_sorted[[" Destination Port", " Protocol"]].head(10))


Sorted dataset preview:
         Destination Port  Protocol
2352942           65535.0       6.0
1936697           65534.0       6.0
2036661           65534.0       6.0
2229746           65534.0       6.0
2413814           65534.0       6.0
1692406           65533.0       6.0
1936313           65533.0       6.0
2036664           65533.0       6.0
2305095           65533.0       6.0
1695579           65532.0       6.0


In [ ]:
#finds unique numbers
unique_ips = df[" Protocol"].unique()
print("Unique Protocols:")
for ip in unique_ips:
    print(ip)

Unique Protocols:
6.0
17.0
0.0
nan


In [ ]:

#First column is row index
df_protocol_sorted = df.sort_values(by=[" Protocol"], ascending=False)
print(df_protocol_sorted[[" Protocol"]].head(10)) #index


df_protocol_sorted = df.sort_values(by=[" Destination Port"], ascending=False)
print(df_protocol_sorted[[" Destination Port"]].head(10))

         Protocol
3119313      17.0
3119315      17.0
3119316      17.0
3119328      17.0
3117590      17.0
3117591      17.0
3117594      17.0
3117595      17.0
3117596      17.0
3117597      17.0
         Destination Port
2352942           65535.0
2036661           65534.0
2229746           65534.0
2413814           65534.0
1936697           65534.0
1692406           65533.0
2036664           65533.0
2305095           65533.0
1936313           65533.0
2719151           65532.0


In [ ]:
#addind/deleting columns

# Add a new column that is Destination Port * 2
df["Port_Double"] = df[" Destination Port"] * 2

# Add a column cycling through 1–10
df["Fold"] = (df.index % 10) + 1

print(df[[" Protocol", " Destination Port", "Fold"]].head(20))


     Protocol   Destination Port  Fold
0         6.0               80.0     1
1         6.0              389.0     2
2         6.0               88.0     3
3         6.0              389.0     4
4         6.0               88.0     5
5         6.0              389.0     6
6         6.0               88.0     7
7         6.0               80.0     8
8         6.0            49666.0     9
9         6.0            49413.0    10
10        6.0               88.0     1
11        6.0              389.0     2
12        6.0             3268.0     3
13        6.0              443.0     4
14        6.0              443.0     5
15        6.0            49666.0     6
16        6.0              135.0     7
17        6.0            49666.0     8
18        6.0              135.0     9
19        6.0            49441.0    10


# Randomization

In [ ]:
#randomization for stratification
#df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True) #random state 42 will make results random but repetitive might be good for testing before we go full random

#This code should work however too much memory to execute

n = len(df)
folds = np.tile(np.arange(1, 11), int(np.ceil(n/10)))[:n]
np.random.shuffle(folds)   # randomize order *in place*
df["Fold"] = folds
print(df[[" Protocol", " Destination Port", " Label", "Fold"]].head(20))



#check fold counts to make sure equal amount randomzied
fold_counts = df["Fold"].value_counts().sort_index()
print(fold_counts)



     Protocol   Destination Port   Label  Fold
0         6.0               80.0  BENIGN     5
1         6.0              389.0  BENIGN     3
2         6.0               88.0  BENIGN     6
3         6.0              389.0  BENIGN     6
4         6.0               88.0  BENIGN    10
5         6.0              389.0  BENIGN     1
6         6.0               88.0  BENIGN     3
7         6.0               80.0  BENIGN     3
8         6.0            49666.0  BENIGN    10
9         6.0            49413.0  BENIGN     7
10        6.0               88.0  BENIGN     4
11        6.0              389.0  BENIGN    10
12        6.0             3268.0  BENIGN     5
13        6.0              443.0  BENIGN     2
14        6.0              443.0  BENIGN     1
15        6.0            49666.0  BENIGN     1
16        6.0              135.0  BENIGN     2
17        6.0            49666.0  BENIGN     1
18        6.0              135.0  BENIGN    10
19        6.0            49441.0  BENIGN     9
Fold
1     31

In [ ]:
#Not randomized
#
# Add a new column that is Destination Port * 2
df["Port_Double"] = df[" Destination Port"] * 2

# Add a column cycling through 1–10
df["Fold"] = (df.index % 10) + 1

print(df[[" Protocol", " Destination Port", "Port_Double"," Label", "Fold"]].head(20))

     Protocol   Destination Port  Port_Double   Label  Fold
0         6.0               80.0        160.0  BENIGN     1
1         6.0              389.0        778.0  BENIGN     2
2         6.0               88.0        176.0  BENIGN     3
3         6.0              389.0        778.0  BENIGN     4
4         6.0               88.0        176.0  BENIGN     5
5         6.0              389.0        778.0  BENIGN     6
6         6.0               88.0        176.0  BENIGN     7
7         6.0               80.0        160.0  BENIGN     8
8         6.0            49666.0      99332.0  BENIGN     9
9         6.0            49413.0      98826.0  BENIGN    10
10        6.0               88.0        176.0  BENIGN     1
11        6.0              389.0        778.0  BENIGN     2
12        6.0             3268.0       6536.0  BENIGN     3
13        6.0              443.0        886.0  BENIGN     4
14        6.0              443.0        886.0  BENIGN     5
15        6.0            49666.0      99